# Train a Convolutional Variational Autoencoder 

Using the Conv-VAE package to train a convolutional variational autoencoder on training data collected from the robot. This is put into a notebook, as it needs to be run only once.

The result of running this code is the creation of a "final model". 

The encoder portion of this model is the component that can be used for the encoder of the robot. 

In [ ]:
import sys
sys.path.append("..")

from settings import Config

import pathlib

# adding the Julian-8897-Conv-VAE-PyTorch into the path
sys.path.append(Config()["conv_vae"]["code_dir"])

# import argparse
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import ImageGrid
#Fixes PosixPath Error

# Oh, this hack was fixing something, but for me it is the other way around
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

from conv_vae import get_conv_vae_config, create_configured_vae_json, train,  latest_training_run

## Training.

The parameters of the training process are set up in the config file.

This will give an error about the pandas, which is due to certain adressing mode in the Conv-Vae util.py, which probably will stop working in pandas 3.0

In [4]:
run = "vae_00"
exp = Config().get_experiment("conv_vae", run)

file = create_configured_vae_json(exp)
config = get_conv_vae_config(file)
# pretty printing the dict behind the config
# print(json.dumps(config.config, indent=4)) 

Missing experiment system dependent config file /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/experiment-config/Tredy2/conv_vae/vae_00_sysdep.yaml, that is ok, proceeding.
Configuration for experiment: conv_vae/vae_00 successfully loaded
/home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/encoding_conv_vae/conv-vae-config-default.json
{'name': 'VAE_Robot', 'n_gpu': 1, 'arch': {'type': 'VanillaVAE', 'args': {'in_channels': 3, 'latent_dims': 128, 'flow': False}}, 'data_loader': {'type': 'CelebDataLoader', 'args': {'data_dir': '/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/conv_vae/vae_00/vae-training-data', 'batch_size': 64, 'shuffle': True, 'validation_split': 0.2, 'num_workers': 2}}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.005, 'weight_decay': 0.0, 'amsgrad': True}}, 'loss': 'elbo_loss', 'metrics': [], 'lr_scheduler': {'type': 'StepLR', 'args': {'step_size': 50, 'gamm

In [5]:
# this is one way to specify the epochs here, but probably not the best
# idea: it should go to the yaml file. 
# 2024-10-16
# I have experienced a mode collapse with the xray training data at 50
config["trainer"]["epochs"] = exp["epochs"] # was 100
config["trainer"]["save_period"] = exp["save_period"] # 5

In [6]:
# actually run the training
print(f'Running the trainer from scratch for {config["trainer"]["epochs"]}')
trainer = train(config)

Running the trainer from scratch for 100


/home/lboloni/Documents/Hackingwork/_Checkouts/Julian-8897-Conv-VAE-PyTorch/Conv-VAE-PyTorch/utils/util.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self._data.total[key] += value * n
/home/lboloni/Documents/Hackingwork/_Checkouts/Ju

In [ ]:
# These are the metrics recorded
# they are of utils/util.py / MetricTracker which has a pandas dataframe as data
print(trainer.train_metrics)
print(trainer.valid_metrics)

# 
trainer.train_metrics._data
# trainer.valid_metrics._data

In [ ]:
trainer.train_metrics._data.shape
!which python

In [ ]:
# FIXME: this one returns models, but it should return what was under it.

model_path = pathlib.Path(config["trainer"]["save_dir"], "models", config["name"])

training_run = latest_training_run(model_path)
print(f'The latest training run is {training_run} in directory\n{config["trainer"]["save_dir"]}')

#print = latest_model

The latest training run is 1220_172927 in directory
/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/conv_vae/vae_00/models
